In [ ]:
# Match table
#------------
# MatchID
# Date
# Patch
# EventID
# EventStage
# Team1ID
# Team2ID
# Team1
# Team2
# Team1_MapScore
# Team2_MapScore

# Game Table
# ------------
# MatchID
# GameID
# Map
# Duration
# Team1ID
# Team2ID
# Team1
# Team2
# Winner
# Team1_TotalRounds
# Team1_SideFirstHalf
# Team1_RoundsFirstHalf
# Team1_RoundsSecondtHalf
# Team1_PistolWon
# Team1_Eco
# Team1_EcoWon
# Team1_SemiEco
# Team1_SemiEcoWon
# Team1_SemiBuy
# Team1_SemiBuyWon
# Team1_FullBuy
# Team1_FullBuyWon
# Team2_TotalRounds
# Team2_SideFirstHalf
# Team2_RoundsFirstHalf
# Team2_RoundsSecondtHalf
# Team2_PistolWon
# Team2_Eco
# Team2_EcoWon
# Team2_SemiEco
# Team2_SemiEcoWon
# Team2_SemiBuy
# Team2_SemiBuyWon
# Team2_FullBuy
# Team2_FullBuyWon


# Game_Rounds
# ------------
# GameID
# Team1ID
# Team2ID
# RoundHistory      NOTE: contains dict of rounds

# format example: 
#    {1: {'RoundWinner': 'AKRE', 'ScoreAfterRound': '1-0', 'WinType': 'defuse', 'Team1Bank': 300.0, 'Team2Bank': 200.0,
#    'Team1BuyType': 'eco', 'Team2BuyType': 'eco'},
#    2: {'RoundWinner': 'AKRE', 'ScoreAfterRound': '2-0', 'WinType': 'elim', 'Team1Bank': 2200.0, 'Team2Bank': 10000.0,
#   'Team1BuyType': 'semi-buy', 'Team2BuyType': 'eco'}, ...


# Game_Scoreboard
# ------------
# GameID
# PlayerID
# TeamID
# PlayerName
# Agent
# Kills
# Deaths
# Assists
# PlusMinus
# KAST_Percent
# ADR
# HS_Percent
# FirstKills
# FirstDeaths
# FKFD_PlusMinus
# Num_2Ks
# Num_3Ks
# Num_4Ks
# Num_5Ks
# OnevOne
# OnevTwo
# OnevThree
# OnevFour
# OnevFive
# Econ
# Plants
# Defuses



In [ ]:
# NOTE: This on

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import traceback
import sqlite3 as sql

pd.set_option('display.max_columns', None)

In [2]:
%cd C:\Users\Joshua Broas\valorant-db

C:\Users\Joshua Broas\valorant-db


In [3]:
conn = sql.connect('valorant.db')

In [4]:
def cleanText(string):
    if string is not None:
        string = string.replace('\n','')
        string = string.replace('\t','')

    return string

def getMatchHeader(source, match_id):
    # Get Event Name and Date
    header = source.find_all("div", class_="match-header-super")[0]
    event = header.find("a", class_='match-header-event')
    a = header.find(href=True)
    event_url = a['href'].lstrip()
    event_id = event_url.split("/")[2]
    event_name = a.div.div.text.lstrip()
    event_stage = a.div.div.find_next_sibling('div').text.lstrip()
    date_header = header.find("div", class_="match-header-date")
    
    date_div = date_header.div
    date = date_div['data-utc-ts']
    
    # Get patch
    patch = None
    try:
        patch = date_header.div.find_next_sibling('div').find_next_sibling('div').text.lstrip()
    except:
        pass
    
    # Get Teams
    match_header_vs = source.find("div", class_="match-header-vs")
    a = match_header_vs.find_all(href=True)
    
    team1_id = a[0]['href'].split("/")[2]
    team2_id = a[1]['href'].split("/")[2]
    team1 = a[0].div.div.text.lstrip()
    team2 = a[1].div.div.text.lstrip()
    
    # Get Score
    match_score = source.find("div", class_="js-spoiler")
    team1_mapscore = match_score.span.text.lstrip()
    team2_mapscore = match_score.span.find_next_sibling('span').find_next_sibling('span').text.lstrip()
    
    # Clean text
    
    event_stage = cleanText(event_stage)
    event_name = cleanText(event_name)
    patch = cleanText(patch)
    team1 = cleanText(team1)
    team2 = cleanText(team2)
    team1_mapscore = cleanText(team1_mapscore)
    team2_mapscore = cleanText(team2_mapscore)
    
    res = {
        'MatchID': match_id,
        'Date': date,
        'Patch': patch,
        'EventID': event_id,
        'EventName': event_name,
        'EventStage': event_stage,
        'Team1ID': int(team1_id),
        'Team2ID': int(team2_id),
        'Team1': team1,
        'Team2': team2,
        'Team1_MapScore': int(team1_mapscore),
        'Team2_MapScore': int(team2_mapscore)
    }
    
    return res

In [5]:
def extractEconomyRoundByRound(cols):
    
    ecoDict = {
        '':'eco',
        '$':'semi-eco',
        '$$':'semi-buy',
        '$$$': 'full-buy'
    }
    
    ecoRoundByRound = {}
    
    for col in cols:
        check = col.find('div', class_='ge-text-light round-num')
        if check is not None:
            t1_buytype = cleanText(col.div.find_next_sibling('div').find_next_sibling('div').text)
            t2_buytype = cleanText(col.div.find_next_sibling('div').find_next_sibling('div').find_next_sibling('div').text)
            roundNum = cleanText(check.text)
            bank = col.find_all('div', class_='bank')
            bankt1 = cleanText(bank[0].text)
            bankt2 = cleanText(bank[1].text)
            
            bankt1 = float(bankt1[:-1])*1000
            bankt2 = float(bankt2[:-1])*1000
            
            
            ecoRoundByRound[int(roundNum)] = {
                'Team1Bank': bankt1,
                'Team2Bank': bankt2,
                'Team1BuyType': ecoDict[t1_buytype],
                'Team2BuyType': ecoDict[t2_buytype]
                }
    
    return ecoRoundByRound

def extractEconomyRows(row, game_id):
    pistolWon_t1 = cleanText(row[0].text)
    eco_list_t1 = cleanText(row[1].text).replace('(', ' ').replace(')',' ').split()
    semi_eco_list_t1 = cleanText(row[2].text).replace('(', ' ').replace(')',' ').split()
    semi_buy_list_t1 = cleanText(row[3].text).replace('(', ' ').replace(')',' ').split()
    full_buy_list_t1 = cleanText(row[4].text).replace('(', ' ').replace(')',' ').split()
    
    pistolWon_t2 = cleanText(row[5].text)
    eco_list_t2 = cleanText(row[6].text).replace('(', ' ').replace(')',' ').split()
    semi_eco_list_t2 = cleanText(row[7].text).replace('(', ' ').replace(')',' ').split()
    semi_buy_list_t2 = cleanText(row[8].text).replace('(', ' ').replace(')',' ').split()
    full_buy_list_t2 = cleanText(row[9].text).replace('(', ' ').replace(')',' ').split()
    
    res = {
        'GameID': game_id,
        'Team1_PistolWon': int(pistolWon_t1),
        'Team1_Eco': int(eco_list_t1[0]),
        'Team1_EcoWon': int(eco_list_t1[1]),
        'Team1_SemiEco': int(semi_eco_list_t1[0]),
        'Team1_SemiEcoWon': int(semi_eco_list_t1[1]),
        'Team1_SemiBuy': int(semi_buy_list_t1[0]),
        'Team1_SemiBuyWon': int(semi_buy_list_t1[1]),
        'Team1_FullBuy': int(full_buy_list_t1[0]),
        'Team1_FullBuyWon': int(full_buy_list_t1[1]),
        'Team2_PistolWon': int(pistolWon_t2),
        'Team2_Eco': int(eco_list_t2[0]),
        'Team2_EcoWon': int(eco_list_t2[1]),
        'Team2_SemiEco': int(semi_eco_list_t2[0]),
        'Team2_SemiEcoWon': int(semi_eco_list_t2[1]),
        'Team2_SemiBuy': int(semi_buy_list_t2[0]),
        'Team2_SemiBuyWon': int(semi_buy_list_t2[1]),
        'Team2_FullBuy': int(full_buy_list_t2[0]),
        'Team2_FullBuyWon': int(full_buy_list_t2[1]),
    }    
    return res

    
def getEconomy(url, game_id):
    full_url = url + '?game=' + game_id[0] + '&tab=economy'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    economy_dict_list = []
    economy_rbr_dict_list = {}
    
    container = soup.find_all('table', class_='wf-table-inset mod-econ')

    idx_1=0
    idx_2=0
    for table in container:
        data_points = table.find_all('div', class_='stats-sq')
        if len(data_points) > 0 and idx_1 <= len(game_id)-1:
            economy_dict_list.append(extractEconomyRows(data_points, game_id[idx_1]))
            idx_1 = idx_1 + 1
        else:
            if idx_2 <= len(game_id)-1:
                extractTD = table.find_all('td')
                economy_rbr_dict_list[game_id[idx_2]] = extractEconomyRoundByRound(extractTD)
                idx_2 = idx_2 + 1
    
    return economy_dict_list, economy_rbr_dict_list

In [6]:
def checkIfEmpty(html):
    split = html.text.split()
    if len(split) == 0:
        return 0
    else:
        return int(split[0])
    

def extractRowPreformanceData(row, game_id):
    player_team = row[0].text.split()
    player = player_team[0]
    team = player_team[1]
    
    agent_pic = row[1].find('img')['src']
    agent = agent_pic.split('/')[5].replace('.png', '')
    
    num_2Ks = checkIfEmpty(row[2])
    num_3Ks = checkIfEmpty(row[3])
    num_4Ks = checkIfEmpty(row[4])
    num_5Ks = checkIfEmpty(row[5])
    OnevOne = checkIfEmpty(row[6])
    OnevTwo = checkIfEmpty(row[7])
    OnevThree = checkIfEmpty(row[8])
    OnevFour = checkIfEmpty(row[9])
    OnevFive = checkIfEmpty(row[10])
    econ = checkIfEmpty(row[11])
    plants = checkIfEmpty(row[12])
    defuses = checkIfEmpty(row[13])

    res = {
        'GameID': game_id,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': agent,
        'Num_2Ks': num_2Ks,
        'Num_3Ks': num_3Ks,
        'Num_4Ks': num_4Ks,
        'Num_5Ks': num_5Ks,
        'OnevOne': OnevOne,
        'OnevTwo': OnevTwo,
        'OnevThree': OnevThree,
        'OnevFour': OnevFour,
        'OnevFive': OnevFive,
        'Econ': econ,
        'Plants': plants,
        'Defuses': defuses
    }
    
    return res


def getPerformanceData(url, game_id):
    full_url = url + '?game=' + game_id[0] + '&tab=performance'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        containers_all = soup.find_all('table', class_='wf-table-inset mod-adv-stats')
        containers_all.pop(0)
    except:
        return []
    
    player_stats = []
    for idx, container in enumerate(containers_all):
        rows = container.find_all('tr')
        for row in rows:
            team = row.find('div', class_='team')
            if team is None:
                continue
            data_points = row.find_all('td')
            row_data = extractRowPreformanceData(data_points, game_id[idx])
            player_stats.append(row_data)
            
    return player_stats

In [22]:
def returnEmptyRow(row):
    name_team_list = cleanText(row[0].text).split()
    if len(name_team_list) == 2:
        player = name_team_list[0]
        team = name_team_list[1]
    else:
        player = name_team_list[0]
        team = ''
        
    res = {
        'GameID': None,
        'PlayerID': None,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': None,
        'ACS': None,
        'Kills': None,
        'Deaths': None,
        'Assists': None,
        'PlusMinus': None,
        'KAST_Percent': None,
        'ADR': None,
        'HS_Percent': None,
        'FirstKills': None,
        'FirstDeaths': None,
        'FKFD_PlusMinus': None
    }
    
    return res


def extractRowData(row):
    name_team_list = cleanText(row[0].text).split()
    if len(name_team_list) == 2:
        player = name_team_list[0]
        team = name_team_list[1]
    else:
        player = name_team_list[0]
        team = ''
    
    combat_score = cleanText(row[2].text)
    try:
        combat_score = int(combat_score)
    except:
        combat_score = None
    kills =  cleanText(row[3].text)
    try:
        kills = int(kills)
    except:
        kills = None
    deaths_unformatted =  cleanText(row[4].text) # need to clean slashes later
    assists =  cleanText(row[5].text)
    try:
        assists = int(assists)
    except:
        assists = None
    plus_minus_unformatted =  cleanText(row[6].text) # need to clean + later
    kast_unformatted =  cleanText(row[7].text)
    adr =  cleanText(row[8].text)
    try:
        adr = int(adr)
    except:
        adr = None
    hs_percent_unformatted =  cleanText(row[9].text)
    first_kills =  cleanText(row[10].text)
    try:
        first_kills = int(first_kills)
    except:
        first_kills = None
    first_deaths =  cleanText(row[11].text)
    try:
        first_deaths = int(first_deaths)
    except:
        first_deaths = None
    fkfd_plus_minus_unformatted = cleanText(row[12].text)
    
    
    # Now onto formatting some of the fields...
    deaths = deaths_unformatted.replace('/','')
    try:
        deaths = int(deaths)
    except:
        deaths = None
    plus_minus = plus_minus_unformatted.replace('+', '')
    try:
        plus_minus = int(plus_minus)
    except:
        plus_minus = None
    kast = kast_unformatted.replace('%','')
    try:
        kast = int(kast)/100 # convert % to decimal
    except:
        kast = None
    hs_percent = hs_percent_unformatted.replace('%','')
    try:
        hs_percent = int(hs_percent)/100 # convert % to decimal
    except:
         hs_percent = None
    fkfd_plus_minus = fkfd_plus_minus_unformatted.replace('+', '')
    try:
        fkfd_plus_minus = int(fkfd_plus_minus)
    except:
        fkfd_plus_minus = None
    
    res = {
        'GameID': None,
        'PlayerID': None,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': None,
        'ACS': combat_score,
        'Kills': kills,
        'Deaths': deaths,
        'Assists': assists,
        'PlusMinus': plus_minus,
        'KAST_Percent': kast,
        'ADR': adr,
        'HS_Percent': hs_percent,
        'FirstKills': first_kills,
        'FirstDeaths': first_deaths,
        'FKFD_PlusMinus': fkfd_plus_minus
    }
    
    return res
    

def getScoreboard(url, game_id, team1_id, team2_id, match_id):
    full_url = url + '?game=' + game_id[0] + '&tab=overview'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    def getScoreboardHeader(soup, match_id):
        # Scoreboard Header
        game_json_list = []
        stats_header = soup.find_all('div', class_='vm-stats-game-header')
        for idx, tab in enumerate(stats_header):
            team_list = tab.find_all('div', class_='team-name')
            team1 = cleanText(team_list[0].text)
            team2 = cleanText(team_list[1].text)    
            score_list = tab.find_all('div', class_='score')
            team1_score = int(cleanText(score_list[0].text))
            team2_score = int(cleanText(score_list[1].text))

            if team1_score > team2_score:
                winner = team1
            else:
                winner = team2

            half_score_list = tab.find_all('span')
            text_list = []
            for txt in half_score_list:
                text_list.append(cleanText(txt.text))
            
            if 'PICK' in text_list:
                # OT
                if len(half_score_list) == 8:
                    team1_firsthalf_score = half_score_list[0].text
                    team1_secondhalf_score = half_score_list[1].text
                    team1_roundsOT = int(half_score_list[2].text)
                    map_name = cleanText(half_score_list[3].text)
                    team2_firsthalf_score = half_score_list[5].text
                    t2_index = 5
                    team2_secondhalf_score = half_score_list[6].text
                    team2_roundsOT = int(half_score_list[7].text)
                    
                else:
                    team1_firsthalf_score = half_score_list[0].text
                    team1_secondhalf_score = half_score_list[1].text
                    map_name = cleanText(half_score_list[2].text)
                    team2_firsthalf_score = half_score_list[4].text
                    t2_index = 4
                    team2_secondhalf_score = half_score_list[5].text
                    team1_roundsOT = 0
                    team2_roundsOT = 0
                
            else:
                #NoOT
                if len(half_score_list) <= 5:
                    team1_firsthalf_score = half_score_list[0].text
                    team1_secondhalf_score = half_score_list[1].text
                    map_name = cleanText(half_score_list[2].text)
                    team2_firsthalf_score = half_score_list[3].text
                    t2_index = 3
                    team2_secondhalf_score = half_score_list[4].text
                    team1_roundsOT = 0
                    team2_roundsOT = 0
                
                else:
                    team1_firsthalf_score = half_score_list[0].text
                    team1_secondhalf_score = half_score_list[1].text
                    team1_roundsOT = int(half_score_list[2].text)
                    map_name = cleanText(half_score_list[3].text)
                    team2_firsthalf_score = half_score_list[4].text
                    t2_index = 4
                    team2_secondhalf_score = half_score_list[5].text
                    team2_roundsOT = int(half_score_list[6].text)

            if 'mod-ct' in half_score_list[0]['class']:
                team1_firsthalf_side = 'defend'
            else:
                team1_firsthalf_side = 'attack'

            if 'mod-ct' in half_score_list[t2_index]['class']:
                team2_firsthalf_side = 'defend'
            else:
                team2_firsthalf_side = 'attack'

            duration_div = tab.find('div', class_='map-duration ge-text-light')
            duration = cleanText(duration_div.text)

            game_json = {
                'GameID': game_id[idx],
                'MatchID': match_id,
                'Map': map_name,
                'Team1ID': team1_id,
                'Team2ID': team2_id,
                'Team1': team1,
                'Team2': team2,
                'Winner': winner,
                'Team1_TotalRounds': team1_score,
                'Team2_TotalRounds': team2_score,
                'Team1_SideFirstHalf': team1_firsthalf_side,
                'Team2_SideFirstHalf': team2_firsthalf_side,
                'Team1_RoundsFirstHalf': int(team1_firsthalf_score),
                'Team1_RoundsSecondtHalf': int(team1_secondhalf_score),
                'Team1_RoundsOT': team1_roundsOT,
                'Team2_RoundsFirstHalf': int(team2_firsthalf_score),
                'Team2_RoundsSecondtHalf': int(team2_secondhalf_score),
                'Team2_RoundsOT': team2_roundsOT
            }
            
            game_json_list.append(game_json)

        return game_json_list
    
    def getRoundByRound(soup, game_id):
        # Round by round
        rounds_container_all = soup.find_all('div', class_='vlr-rounds-row')
        round_by_round_dict = {}
        
        subtract_idx = 0
        for idx, rounds_container in enumerate(rounds_container_all):
            teams = rounds_container.find_all('div', class_='team')
            team1_abr = cleanText(teams[0].text)
            team2_abr = cleanText(teams[1].text)
            cols = rounds_container.find_all('div', class_='vlr-rounds-row-col')
            cols.pop(0)
                
            round_by_round = {}

            team1_start_side_flag = False
            team1_start_side = ''
            for col in cols:
                rnd_number = col.find('div', class_='rnd-num')
                if len(col['class']) == 1: #Length will be 2 with: 'vlr-rounds-row-col mod-spacing'
                    round_number = int(cleanText(rnd_number.text))
                    class_name = col.div.find_next_sibling('div')['class']
                    if 'mod-win' in class_name:
                        if team1_start_side_flag is False:
                            if 'mod-ct' in class_name:
                                team1_start_side = 'defend'
                                team1_start_side_flag = True
                            if 'mod-t' in class_name:
                                team1_start_side = 'attack'
                                team1_start_side_flag = True

                        win_type_img = col.find('img')
                        win_type = win_type_img['src'].split("/")[5].replace('.webp','')
                        curr_score_tag = col.find('div', class_='rnd-currscore')
                        current_score = cleanText(curr_score_tag.text)
                        round_by_round[round_number] = {
                            'RoundWinner': team1_abr,
                            'ScoreAfterRound': current_score,
                            'WinType': win_type,
                        }

                    else: # Team 2 won OR no rounds were played (game ended)
                        row2 = col.div.find_next_sibling('div').find_next_sibling('div')
                        row2_class = row2['class']
                        if 'mod-win' in row2_class:
                            win_type_img = row2.find('img')
                            win_type = win_type_img['src'].split("/")[5].replace('.webp','')
                            curr_score_tag = col.find('div', class_='rnd-currscore')
                            current_score = cleanText(curr_score_tag.text)
                            round_by_round[round_number] = {
                                'RoundWinner': team2_abr,
                                'ScoreAfterRound': current_score,
                                'WinType': win_type,
                            }

                        else:
                            continue
            if len(cols) < 20:
                needUpdate = round_by_round_dict[game_id[idx-subtract_idx-1]]
                needUpdate.update(round_by_round)
                #print(game_id)
                #print('id', idx-1)
                #print(game_id[idx-1])
                round_by_round_dict[game_id[idx-subtract_idx-1]] = needUpdate
                subtract_idx = subtract_idx + 1
                continue

            idx = idx - subtract_idx

            round_by_round_dict[game_id[idx]] = round_by_round
                
        return round_by_round_dict
    
    # Get Player Stats
    def getPlayerStats(soup):
        containers_all = soup.find_all('div', class_='vm-stats-game')
        
        checkIfNoStats = soup.find('div', class_='rnd-sq mod-win mod-ct')
        missingData = False
        if checkIfNoStats is None:
            missingData = True
            
        containers = []
        for check in containers_all: # Get rid of overall tab
            header = check.find('div', class_='vm-stats-game-header')
            if header is not None:
                containers.append(check)
            
        player_stats = []
        for idx, container in enumerate(containers):
            container = container.find_all('table', class_='wf-table-inset mod-overview')
            for table in container:
                rows = table.find_all('tr')
                for row in rows:
                    data_points = row.find_all('td')
                    agent_img = row.find('img')
                    player_id_search = row.find('a')

                    # There's times where it gets the headers which we don't want
                    if (len(data_points) > 0):
                        if missingData is True:
                            agent = ''
                            row_data = returnEmptyRow(data_points)
                        else:
                            try:
                                agent = agent_img['alt']
                            except:
                                agent = ''
                            row_data = extractRowData(data_points)
                        
                        try:
                            player_id = player_id_search['href'].split('/')[2]
                        except:
                            player_id = None
                            
                        row_data = extractRowData(data_points)
                        row_data['GameID'] = game_id[idx]
                        row_data['Agent'] = agent
                        row_data['PlayerID'] = player_id

                        player_stats.append(row_data)
        return player_stats
        
    try:
        game_dict = getScoreboardHeader(soup, match_id)
    except:
        print('Unable to execute getScoreboardHeader')
        traceback.print_exc()
    try:
        rbr_dict = getRoundByRound(soup, game_id)
    except:
        print('Unable to execute getRoundByRound')
        traceback.print_exc()
    try:
        player_stats_dict = getPlayerStats(soup)
    except:
        print('Unable to execute getPlayerStats')
        traceback.print_exc()
    
    return game_dict, rbr_dict, player_stats_dict

                
                
def getMapStats(source, url, team1_id, team2_id, match_id):
    header = source.find("div", class_="vm-stats")
    
    games = header.find_all("div", class_="vm-stats-gamesnav-item js-map-switch")
    
    game_id = []
    for game in games:
        game_id.append(game['data-game-id'])
        
    # Need to handle case if its only a single game
    if len(game_id) == 0:
        game_id.append(header['data-game-id'])
    
    game_dict_list, rbr_dict_list, player_stats_dict_list = getScoreboard(url, game_id, team1_id, team2_id, match_id)
    
    return game_dict_list, rbr_dict_list, player_stats_dict_list, game_id


In [26]:
ext = '/62393/booster-seat-gaming-vs-pho-real-nerd-street-gamers-winter-championship-regional-lan-philadelphif'
url = 'https://www.vlr.gg' + ext
#url = 'https://www.vlr.gg/61393/akrew-vs-super-squad-knights-monthly-gauntlet-december-ro16'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

match_row = getMatchHeader(soup, '1')

if match_row['Team1_MapScore'] == 0 and match_row['Team1_MapScore'] == 0:
    print('doesnt work')
match_df = pd.DataFrame(match_row, index=[0])
team1_id = match_row['Team1ID']
team2_id = match_row['Team2ID']

game_dict_list, rbr_dict_list, player_stats_dict_list, game_id_list = getMapStats(soup, url, team1_id, team2_id, '1')
adv_player_stats = getPerformanceData(url, game_id_list)
player_stats_df = pd.DataFrame(player_stats_dict_list)
adv_player_stats_df = pd.DataFrame(adv_player_stats)

#for g in rbr_dict_list:
#    print(rbr_dict_list[g])

if adv_player_stats:
    all_player_stats = player_stats_df.merge(adv_player_stats_df, on=['GameID', 'PlayerName', 'Agent'])
    all_player_stats = all_player_stats.drop(columns=['TeamAbbreviation_y'])
    all_player_stats = all_player_stats.rename(columns={'TeamAbbreviation_x':'TeamAbbreviation'})
else:
    all_player_stats = player_stats_df
    all_player_stats['Num_2Ks'] = None
    all_player_stats['Num_3Ks'] = None
    all_player_stats['Num_4Ks'] = None
    all_player_stats['Num_5Ks'] = None
    all_player_stats['OnevOne'] = None
    all_player_stats['OnevTwo'] = None
    all_player_stats['OnevThree'] = None
    all_player_stats['OnevFour'] = None
    all_player_stats['OnevFive'] = None
    all_player_stats['Econ'] = None
    all_player_stats['Plants'] = None
    all_player_stats['Defuses'] = None
    all_player_stats['Econ'] = None


economy_dict_list, economy_rbr_dict_list = getEconomy(url, game_id_list)

if len(economy_dict_list) > 0:
    game_df = pd.DataFrame(game_dict_list)
    econ_df = pd.DataFrame(economy_dict_list)
    full_game_df = game_df.merge(econ_df, on=['GameID'])

    game_rounds = []
    for key in rbr_dict_list:
        econ_info = economy_rbr_dict_list[key]
        for idx, round_num in enumerate(rbr_dict_list[key]):
            rbr_dict_list[key][round_num].update(econ_info[round_num])
            
        res = {
            'GameID': key,
            'Team1ID': team1_id,
            'Team2ID': team2_id,
            'RoundHistory': str(rbr_dict_list[key])
        }
        game_rounds.append(res)

    game_rounds_df = pd.DataFrame(game_rounds)
else:
    full_game_df = pd.DataFrame(game_dict_list)
    game_rounds = []
    for game in game_id_list:
        res = {
            'GameID': game,
            'Team1ID': team1_id,
            'Team2ID': team2_id,
            'RoundHistory': None
        }
        game_rounds.append(res)
    game_rounds_df = pd.DataFrame(game_rounds)
    
# match_df Match Table
# full_game_df Game Table
# game_rounds_df Game_Rounds Table
# all_player_stats Game_Scoreboard Table

all_player_stats


,GameID,PlayerID,PlayerName,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,KAST_Percent,ADR,HS_Percent,FirstKills,FirstDeaths,FKFD_PlusMinus,Num_2Ks,Num_3Ks,Num_4Ks,Num_5Ks,OnevOne,OnevTwo,OnevThree,OnevFour,OnevFive,Econ,Plants,Defuses
0,60894,8419,Reduxx,Boos,jett,313,24,10,3,14,0.65,195,0.31,4,4,0,2,2,2,0,1,0,0,0,0,74,0,0
1,60894,466,ChurmZ,Boos,chamber,227,16,10,7,6,0.90,161,0.16,1,1,0,3,1,0,0,0,0,0,0,0,67,2,0
2,60894,3712,diaamond,Boos,sova,226,17,9,8,8,0.85,148,0.27,3,0,3,1,2,0,0,1,0,0,0,0,58,3,0
3,60894,5099,Boltzy,Boos,viper,218,17,12,2,5,0.70,141,0.19,3,0,3,3,1,0,0,1,0,0,0,0,48,0,0
4,60894,3983,Virtyy,Boos,skye,80,5,13,3,-8,0.70,55,0.22,3,1,2,1,0,0,0,0,0,0,0,0,21,0,0
5,60894,16757,Cohburg,Pho,astra,229,15,16,5,-1,0.55,155,0.31,1,2,-1,3,2,0,0,0,0,0,0,0,62,1,0
6,60894,19293,evaD,Pho,sova,203,12,14,6,-2,0.65,151,0.22,0,0,0,1,1,0,0,0,1,0,0,0,73,1,0
7,60894,19291,Bobovo,Pho,chamber,173,12,16,2,-4,0.70,103,0.18,1,3,-2,2,0,0,0,0,0,0,0,0,42,0,0
8,60894,1080,Dopai,Pho,skye,125,9,15,1,-6,0.50,94,0.36,1,3,-2,0,0,1,0,0,0,0,0,0,42,0,0
9,60894,21542,Zento,Pho,viper,89,6,18,3,-12,0.45,57,0.22,3,6,-3,1,0,0,0,0,0,0,0,0,21,2,0


In [27]:
try:
    match_df.to_sql('Matches', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Match Table')
    traceback.print_exc()
    pass

try:
    full_game_df.to_sql('Games', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game Table')
    traceback.print_exc()
    pass
try:
    game_rounds_df.to_sql('Game_Rounds', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game_Rounds Table')
    traceback.print_exc()
    pass
try:
    all_player_stats.to_sql('Game_Scoreboard', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game_Scoreboard Table')
    traceback.print_exc()
    pass

print('done!')

done!


In [9]:
# Get all the urls
test_list = []
for i in range(1, 240):
    url = f'https://www.vlr.gg/matches/results/?page={i}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    match_container = soup.find('div', class_='col-container')
    urls = match_container.find_all('a')

    for url in urls:
        verify_match = url['href'].split("/")
        if 'matches' not in verify_match:
            test_list.append((int(verify_match[1]), url['href']))


In [32]:
l1 = test_list[0:2000]
l2 = test_list[2000:4000]
l3 = test_list[4000:6000]
l4 = test_list[6000:8000]
l5 = test_list[8000:10000]
l6 = test_list[10000:]

In [33]:
print(len(l6))

1917


In [17]:
failed_list = []

In [34]:
for index, combo in enumerate(l6):
    print(index)
    try:
        url = 'https://www.vlr.gg' + combo[1]
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        try:
            url = 'https://www.vlr.gg' + combo[1]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
        except:
            failed_list.append(combo)
            continue
            
    try:
        try:
            match_row = getMatchHeader(soup, combo[0])
            if match_row['Team1_MapScore'] == 0 and match_row['Team1_MapScore'] == 0:
                continue
            match_df = pd.DataFrame(match_row, index=[0])
            team1_id = match_row['Team1ID']
            team2_id = match_row['Team2ID']

            game_dict_list, rbr_dict_list, player_stats_dict_list, game_id_list = getMapStats(soup, url, team1_id, team2_id, combo[0])

            if game_id_list is None:
                print('(1) Could not get info for link: ')
                print(combo[1])
                traceback.print_exc()
                failed_list.append(combo)
                continue
        except:
            print('(2) Could not get info for link: ')
            print(combo[1])
            traceback.print_exc()
            failed_list.append(combo)
            continue

        adv_player_stats = getPerformanceData(url, game_id_list)
        player_stats_df = pd.DataFrame(player_stats_dict_list)
        adv_player_stats_df = pd.DataFrame(adv_player_stats)


        if adv_player_stats:
            all_player_stats = player_stats_df.merge(adv_player_stats_df, on=['GameID', 'PlayerName', 'Agent'])
            all_player_stats = all_player_stats.drop(columns=['TeamAbbreviation_y'])
            all_player_stats = all_player_stats.rename(columns={'TeamAbbreviation_x':'TeamAbbreviation'})
        else:
            all_player_stats = player_stats_df
            all_player_stats['Num_2Ks'] = None
            all_player_stats['Num_3Ks'] = None
            all_player_stats['Num_4Ks'] = None
            all_player_stats['Num_5Ks'] = None
            all_player_stats['OnevOne'] = None
            all_player_stats['OnevTwo'] = None
            all_player_stats['OnevThree'] = None
            all_player_stats['OnevFour'] = None
            all_player_stats['OnevFive'] = None
            all_player_stats['Econ'] = None
            all_player_stats['Plants'] = None
            all_player_stats['Defuses'] = None
            all_player_stats['Econ'] = None


        economy_dict_list, economy_rbr_dict_list = getEconomy(url, game_id_list)

        if len(economy_dict_list) > 0:
            game_df = pd.DataFrame(game_dict_list)
            econ_df = pd.DataFrame(economy_dict_list)
            full_game_df = game_df.merge(econ_df, on=['GameID'])

            game_rounds = []
            for key in rbr_dict_list:
                econ_info = economy_rbr_dict_list[key]
                for idx, round_num in enumerate(rbr_dict_list[key]):
                    rbr_dict_list[key][round_num].update(econ_info[round_num])

                res = {
                    'GameID': key,
                    'Team1ID': team1_id,
                    'Team2ID': team2_id,
                    'RoundHistory': str(rbr_dict_list[key])
                }
                game_rounds.append(res)

            game_rounds_df = pd.DataFrame(game_rounds)
        else:
            full_game_df = pd.DataFrame(game_dict_list)
            game_rounds = []
            for game in game_id_list:
                res = {
                    'GameID': game,
                    'Team1ID': team1_id,
                    'Team2ID': team2_id,
                    'RoundHistory': None
            }
            game_rounds.append(res)
        game_rounds_df = pd.DataFrame(game_rounds)
        
        try:
            match_df.to_sql('Matches', conn, index=False, if_exists='append')
            full_game_df.to_sql('Games', conn, index=False, if_exists='append')
            game_rounds_df.to_sql('Game_Rounds', conn, index=False, if_exists='append')
            all_player_stats.to_sql('Game_Scoreboard', conn, index=False, if_exists='append')
        except:
            print(f'****ERROR ADDING TABLE for {combo[1]}******')
            failed_list.append(combo)
            traceback.print_exc()
            continue
    except:
        print('SOME RANDOM ERORR')
        failed_list.append(combo)
        continue


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
Unable to execute getScoreboardHeader
Unable to execute getRoundByRound
Unable to execute getPlayerStats
(2) Could not get info for link: 
/3142/vimdolol-vs-gamelanders-evolution-open-brazil-tournament-grand-final
38


Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 360, in getScoreboard
    game_dict = getScoreboardHeader(soup, match_id)
  File "<ipython-input-22-032db55da057>", line 212, in getScoreboardHeader
    'GameID': game_id[idx],
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 365, in getScoreboard
    rbr_dict = getRoundByRound(soup, game_id)
  File "<ipython-input-22-032db55da057>", line 305, in getRoundByRound
    round_by_round_dict[game_id[idx]] = round_by_round
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 370, in getScoreboard
    player_stats_dict = getPlayerStats(soup)
  File "<ipython-input-22-032db55da057>", line 352, in getPlayerStats
    row_data['GameID'] = game_id[idx]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-34-affff1575b33>", line 25, in <module>
  

39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
(2) Could not get info for link: 
/3167/china-nguyen-vs-luminosity-nerd-street-gamers-september-monthly-gf
61


Traceback (most recent call last):
  File "<ipython-input-34-affff1575b33>", line 18, in <module>
    match_row = getMatchHeader(soup, combo[0])
  File "<ipython-input-4-0018c6b580d0>", line 10, in getMatchHeader
    header = source.find_all("div", class_="match-header-super")[0]
IndexError: list index out of range


(2) Could not get info for link: 
/3165/china-nguyen-vs-team-fyb-nerd-street-gamers-september-monthly-sf
62


...

(2) Could not get info for link: 
/196/inst-vs-tbd-grizicup-playoffs-qf
1912


Traceback (most recent call last):
  File "<ipython-input-34-affff1575b33>", line 18, in <module>
    match_row = getMatchHeader(soup, combo[0])
  File "<ipython-input-4-0018c6b580d0>", line 34, in getMatchHeader
    team2_id = a[1]['href'].split("/")[2]
IndexError: list index out of range


1913
Unable to execute getScoreboardHeader
Unable to execute getRoundByRound
Unable to execute getPlayerStats
(2) Could not get info for link: 
/192/nip-vs-need-more-dm-cooler-cup-playoffs-sf
1914


Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 360, in getScoreboard
    game_dict = getScoreboardHeader(soup, match_id)
  File "<ipython-input-22-032db55da057>", line 212, in getScoreboardHeader
    'GameID': game_id[idx],
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 365, in getScoreboard
    rbr_dict = getRoundByRound(soup, game_id)
  File "<ipython-input-22-032db55da057>", line 305, in getRoundByRound
    round_by_round_dict[game_id[idx]] = round_by_round
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-22-032db55da057>", line 370, in getScoreboard
    player_stats_dict = getPlayerStats(soup)
  File "<ipython-input-22-032db55da057>", line 352, in getPlayerStats
    row_data['GameID'] = game_id[idx]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-34-affff1575b33>", line 25, in <module>
  

1915
1916


In [35]:
print(len(failed_list))

348


In [36]:
print(failed_list)

[(61395, '/61395/bait-academy-vs-team-reckaroo-knights-monthly-gauntlet-december-ro16'), (61444, '/61444/god-particles-vs-reckoning-esports-asus-rog-showdown-series-sf'), (60388, '/60388/vietnamese-curry-vs-froggers-frag-valorant-major-fullerton-qf'), (60301, '/60301/booster-seat-gaming-vs-iwnl-frag-valorant-major-philadelphia-r1'), (55503, '/55503/ex-flickbaiters-vs-andbox-nerd-street-gamers-winter-championship-open-8-ro16'), (51266, '/51266/acend-vs-keyd-stars-valorant-champions-opening-a'), (57281, '/57281/samelol-vs-third-impact-vcs-midwest-showdown-qf'), (51367, '/51367/crimas-disciples-vs-nerdnecks-aen-spring-series-gf'), (51490, '/51490/portuguesa-vs-stars-horizon-copa-rakin-season-3-gf'), (52466, '/52466/evil-geniuses-vs-darkzero-esports-nerd-street-gamers-winter-championship-open-7-ro16'), (51715, '/51715/the-mafia-vs-betrayeds-fantasy-team-rival-1-qf'), (51013, '/51013/paper-rex-vs-alter-ego-the-esports-club-showdown-ubf'), (50794, '/50794/ivy-vs-natus-vincere-moscow-cyberspo

Need to run these SQL commands after to fix maps:


UPDATE Games
SET Map = 'Icebox'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'IceboxPICK'
);

UPDATE Games
SET Map = 'Bind'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'BindPICK'
);

UPDATE Games
SET Map = 'Haven'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'HavenPICK'
);

UPDATE Games
SET Map = 'Split'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'SplitPICK'
);

UPDATE Games
SET Map = 'Breeze'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'BreezePICK'
);

UPDATE Games
SET Map = 'Fracture'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'FracturePICK'
);

UPDATE Games
SET Map = 'Ascent'
WHERE GameID IN (
	SELECT GameID
	FROM Games
	WHERE Map LIKE 'AscentPICK'
);